In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from CARTO.Decoder.data import asdf_dataset
import open3d as o3d
import pathlib
import random 
import numpy as np

In [3]:

def get_colors(n): 
  ret = [] 
  r = int(random.random() * 256) 
  g = int(random.random() * 256) 
  b = int(random.random() * 256) 
  step = 256 / n 
  for i in range(n): 
    r += step 
    g += step 
    b += step 
    r = int(r) % 256 
    g = int(g) % 256 
    b = int(b) % 256 
    ret.append(np.array([r,g,b])/255)  
  return ret 

In [7]:
category = "oven"

local_dataset = asdf_dataset.ASDFDataset(
    [f"examples/splits/sm_{category}_6_angle"],
    subsample_amount=1e12,  # Very big
    load_ram=False,
    train=True,
    load_gt=True
)

all_categories = asdf_dataset.ASDFDataset(
    [
        "examples/splits/sm_door_6_angle", "examples/splits/sm_laptop_6_angle",
        "examples/splits/sm_oven_6_angle", "examples/splits/sm_stapler_6_angle",
        "examples/splits/sm_washing_machine_6_angle"
    ],
    subsample_amount=1e12,  # Very big
    load_ram=False,
    train=True
)

In [8]:
out_path= pathlib.Path("vis/asdf_testing_full_pc") / category
out_path.mkdir(exist_ok=True, parents=True)

In [ ]:
all_pcds = []
N = len(local_dataset) // 6
colors = get_colors(N)

# for idx in range(0, len(local_dataset), 6):
for idx in range(0, 6, 1):
    dp = local_dataset[idx]
    pcd = o3d.geometry.PointCloud()
    # pcd.points = o3d.utility.Vector3dVector(dp.points[dp.sdf_values.abs() < 1e-2])
    pcd.points = o3d.utility.Vector3dVector(dp.full_pc[::3, :])
    pcd.paint_uniform_color(colors[idx // 6])
    print(dp.zero_joint_config)
    o3d.io.write_point_cloud(str(out_path / f"{idx}.ply"), pcd)
    print(pcd.get_min_bound())
    print(pcd.get_max_bound())

In [10]:
# o3d.visualization.draw_plotly(all_pcds)